In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from pathlib import Path
import sys


ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, load_dataset
import numpy as np
import pandas as pd
from template import LLAMA3_CHAT_TEMPLATE
from peft import PeftModel
from config import Config2, Config

In [ ]:
df = pd.read_csv('./data/wpu_data/retain_100.csv')
dataset = Dataset.from_pandas(df)

In [ ]:
# Calculates EL2N scores

def get_el2n_scores(model, tokenizer, question, answer):
    
    templated_question = LLAMA3_CHAT_TEMPLATE.format(question = question)
    prompt = f"{templated_question}{answer}{tokenizer.eos_token}"

    # tokenize without the answer
    inputs_no_answer = tokenizer(templated_question, return_tensors='pt').to(model.device)
    full_input_ids_no_answer = inputs_no_answer.input_ids[0].tolist()

    # tokenize with the answer
    inputs_full = tokenizer(prompt, return_tensors = 'pt').to(model.device)
    full_input_ids_full = inputs_full.input_ids[0].tolist()


    # Find the start index of the answer tokens within the full prompt's token IDs
    start_index = -1
    min_len = min(len(full_input_ids_no_answer), len(full_input_ids_full))
    for i in range(min_len):
        if full_input_ids_no_answer[i] != full_input_ids_full[i]:
            start_index = i
            break

    if start_index == -1 and len(full_input_ids_full) > len(full_input_ids_no_answer):
        start_index = len(full_input_ids_no_answer)
    elif start_index == -1:
        print(f"Warning: Could not find the start of the answer in the prompt for question: {question[:50]}...")
        print("Full Input IDs (Full):", full_input_ids_full)
        print("Full Input IDs (No Answer):", full_input_ids_no_answer)
        return 0.0

    labels = inputs_full.input_ids.clone()
    # Mask out tokens before the start of the answer
    labels[0, :start_index] = -100

    with torch.no_grad():
      outputs = model(**inputs_full, labels = labels)
      logits = outputs.logits

    logits_start_index = max(0, start_index - 1)
    
    logits_for_answer = logits[0, logits_start_index: -1, :]

    labels_for_answer = labels[0, start_index: ]

    valid_label_mask_answer = labels_for_answer != -100
    valid_labels = labels_for_answer[valid_label_mask_answer]

    if valid_labels.numel() == 0:

      print("Warning: No valid labels found in the answer segment.")
      return 0.0
    
    valid_logits = logits_for_answer[valid_label_mask_answer]

    if valid_logits.shape[0] != valid_labels.shape[0]:
        print("Error: Mismatch between the number of valid logits and labels.")
        return 0.0
    
    probs = F.softmax(valid_logits, dim = -1)

    one_hot_labels = F.one_hot(valid_labels, num_classes = probs.shape[-1]).float()

    l2_norms = torch.norm(probs - one_hot_labels, p=2, dim=-1)

    el2n_score = l2_norms.mean().item()

    return el2n_score

In [ ]:
def get_grand_data_top(percent_to_keep_top):
    n_total = len(dataset)
    start_index_top = n_total - int(n_total * percent_to_keep_top)
    grand_top = dataset.select(range(start_index_top, n_total))
    return grand_top

## For WPU

### With Gradient Descent Loss

In [ ]:
model_id = 'path/to/check_point_gradient_descent_model'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map='auto')

In [ ]:
import time

start_time = time.time()
example_scores = []

for example in dataset:
    scores = get_el2n_scores(model, tokenizer, example['question'], example['answer'])
    example_scores.append(scores)


dataset = dataset.add_column('el2n_score', example_scores)
end_time = time.time() 

print(f"Time taken: {end_time - start_time} seconds")

In [ ]:
dataset = dataset.sort('el2n_score')

In [ ]:
def get_grand_data_top(percent_to_keep_top):
    n_total = len(dataset)
    start_index_top = n_total - int(n_total * percent_to_keep_top)
    grand_top = dataset.select(range(start_index_top, n_total))
    return grand_top

In [ ]:
grand_desc_top_50 = get_grand_data_top(0.5)
print(grand_desc_top_50.shape)

grand_desc_top_10 = get_grand_data_top(0.1)
print(grand_desc_top_10.shape)

grand_desc_top_20 = get_grand_data_top(0.2)
print(grand_desc_top_20.shape)

grand_desc_top_5 = get_grand_data_top(0.05)
print(grand_desc_top_5.shape)

grand_desc_top_2 = get_grand_data_top(0.02)
print(grand_desc_top_2.shape)

grand_desc_top_1 = get_grand_data_top(0.01)
print(grand_desc_top_1.shape)

In [ ]:
grand_desc_top_50.to_csv('./data/wpu_data/coresets/grand/grad_descent/grand_desc_top_50.csv', index = False)
grand_desc_top_10.to_csv('./data/wpu_data/coresets/grand/grad_descent/grand_desc_top_10.csv', index = False)
grand_desc_top_20.to_csv('./data/wpu_data/coresets/grand/grad_descent/grand_desc_top_20.csv', index = False)
grand_desc_top_5.to_csv('./data/wpu_data/coresets/grand/grad_descent/grand_desc_top_5.csv', index = False)
grand_desc_top_2.to_csv('./data/wpu_data/coresets/grand/grad_descent/grand_desc_top_2.csv', index = False)
grand_desc_top_1.to_csv('./data/wpu_data/coresets/grand/grad_descent/grand_desc_top_1.csv', index = False)

dataset.to_csv('./data/wpu_data/coresets/grand/grad_descent/grand_desc_scores.csv', index = False)

### With Gradient Difference Loss

In [ ]:
model_id = 'path/to/check_point_gradient_difference_model' #this is a peft model, so we need to concat the adapter. For descent we did it in the finetuning code already.
cfg = Config()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model_id)
base_model = AutoModelForCausalLM.from_pretrained(cfg.model_id, torch_dtype=torch.bfloat16, device_map='auto')
model = PeftModel.from_pretrained(base_model, model_id, device_map='auto', torch_dtype=torch.bfloat16)

In [ ]:
import time

start_time = time.time()

example_scores = []

for example in dataset:
    scores = get_el2n_scores(model, tokenizer, example['question'], example['answer'])
    example_scores.append(scores)


dataset = dataset.add_column('el2n_score_gd', example_scores)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

In [ ]:
dataset = dataset.sort('el2n_score_gd')

In [ ]:
grand_gd_top_50 = get_grand_data_top(0.5)
print(grand_gd_top_50.shape)

grand_gd_top_10 = get_grand_data_top(0.1)
print(grand_gd_top_10.shape)

grand_gd_top_20 = get_grand_data_top(0.2)
print(grand_gd_top_20.shape)

grand_gd_top_5 = get_grand_data_top(0.05)
print(grand_gd_top_5.shape)

grand_gd_top_2 = get_grand_data_top(0.02)
print(grand_gd_top_2.shape)

grand_gd_top_1 = get_grand_data_top(0.01)
print(grand_gd_top_1.shape)

In [ ]:
grand_desc_top_50.to_csv('./data/wpu_data/coresets/grand/grad_diff/grand_diff_top_50.csv', index = False)
grand_desc_top_10.to_csv('./data/wpu_data/coresets/grand/grad_diff/grand_diff_top_10.csv', index = False)
grand_desc_top_20.to_csv('./data/wpu_data/coresets/grand/grad_diff/grand_diff_top_20.csv', index = False)
grand_desc_top_5.to_csv('./data/wpu_data/coresets/grand/grad_diff/grand_diff_top_5.csv', index = False)
grand_desc_top_2.to_csv('./data/wpu_data/coresets/grand/grad_diff/grand_diff_top_2.csv', index = False)
grand_desc_top_1.to_csv('./data/wpu_data/coresets/grand/grad_diff/grand_diff_top_1.csv', index = False)

dataset.to_csv('./data/wpu_data/coresets/grand/grad_diff/grand_diff_scores.csv', index = False)

## Unified data

### gradient difference

In [ ]:
cfg = Config2() #config2 is for mix

In [ ]:
model_id = 'path/to/check_point_gradient_difference_model'
print(cfg.model_id)
tokenizer = AutoTokenizer.from_pretrained(cfg.model_id)
base_model = AutoModelForCausalLM.from_pretrained(cfg.model_id, torch_dtype=torch.bfloat16, device_map='auto')
model = PeftModel.from_pretrained(base_model, model_id, device_map='auto', torch_dtype=torch.bfloat16)

In [ ]:
df = pd.read_csv('./data/mix/full_retain.csv')
dataset = Dataset.from_pandas(df)

In [ ]:
import time

start_time = time.time()

example_scores = []

for example in dataset:
    scores = get_el2n_scores(model, tokenizer, example['question'], example['answer'])
    example_scores.append(scores)


dataset = dataset.add_column('el2n_score_gd', example_scores)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

In [ ]:
dataset = dataset.sort('el2n_score_gd')

In [ ]:
diff_1 = get_grand_data_top(0.01)
print(diff_1.shape)

diff_2 = get_grand_data_top(0.02)
print(diff_2.shape)

diff_5 = get_grand_data_top(0.05)
print(diff_5.shape)

diff_10 = get_grand_data_top(0.1)
print(diff_10.shape)

diff_20 = get_grand_data_top(0.2)
print(diff_20.shape)

In [ ]:
diff_1.to_csv('./data/mix/coresets/grand/grad_difference/diff_1.csv', index = False)
diff_2.to_csv('./data/mix/coresets/grand/grad_difference/diff_2.csv', index = False)
diff_5.to_csv('./data/mix/coresets/grand/grad_difference/diff_5.csv', index = False)
diff_10.to_csv('./data/mix/coresets/grand/grad_difference/diff_10.csv', index = False)
diff_20.to_csv('./data/mix/coresets/grand/grad_difference/diff_20.csv', index = False)

### With Gradient Descent

In [ ]:
model_id = 'path/to/check_point_gradient_descent_model'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map='auto')

In [ ]:
df = pd.read_csv('./data/mix/full_retain.csv')
dataset = Dataset.from_pandas(df)

In [ ]:
import time

start_time = time.time()

example_scores = []

for example in dataset:
    scores = get_el2n_scores(model, tokenizer, example['question'], example['answer'])
    example_scores.append(scores)


dataset = dataset.add_column('el2n_score_gd', example_scores)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} seconds")

In [ ]:
dataset = dataset.sort('el2n_score_gd')

In [ ]:
desc_1 = get_grand_data_top(0.01)
print(desc_1.shape)

desc_2 = get_grand_data_top(0.02)
print(desc_2.shape)

desc_5 = get_grand_data_top(0.05)
print(desc_5.shape)

desc_10 = get_grand_data_top(0.1)
print(desc_10.shape)

desc_20 = get_grand_data_top(0.2)
print(desc_20.shape)

In [ ]:
desc_1.to_csv('./data/mix/coresets/grand/grad_descent/desc_1.csv', index = False)
desc_2.to_csv('./data/mix/coresets/grand/grad_descent/desc_2.csv', index = False)
desc_5.to_csv('./data/mix/coresets/grand/grad_descent/desc_5.csv', index = False)
desc_10.to_csv('./data/mix/coresets/grand/grad_descent/desc_10.csv', index = False)
desc_20.to_csv('./data/mix/coresets/grand/grad_descent/desc_20.csv', index = False)